In [33]:
import pandas as pd
import numpy as np
import math
import os
from sqlalchemy import create_engine
import geopandas as gp
import json
import altair as alt
import requests

In [10]:
%load_ext dotenv
%dotenv ../.env

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [11]:
cnx = create_engine(os.getenv('EDMDB'))

Test connection

In [12]:
df2000 = pd.read_sql_query('SELECT * FROM gep690.census_2000', cnx)
df2000.head()

,id,geom,borocode,boro_ct2000,boro_name,cdeligibil,ct2000,ctlabel,ntacode,ntaname,...,median_hh_income,tot_housing_units,owner_occupied_housing_units,pct_owner_occupied,median_housing_value,in_labor_force_16up,in_labor_force_unemployed_16up,pct_unemployed_16up,pct_below_poverty_level,pct_educated
0,1330,0106000020D70800000100000001030000000100000064...,SI,5029104,Staten Island,I,029104,291.04,SI05,New Springville-Bloomfield-Travis,...,62260,2355,1658,70.4,187500,3705,286,5.3,7.220,25.687
1,1781,0106000020D7080000010000000103000000010000002A...,SI,5008900,Staten Island,I,008900,89,SI22,West New Brighton-New Brighton-St. George,...,0,0,0,0.0,0,0,0,0.0,0.000,0.000
2,38,0106000020D70800000100000001030000000100000013...,BX,2025000,Bronx,I,025000,250,BX49,Pelham Parkway,...,38854,996,415,41.7,194100,998,45,2.3,6.673,24.171
3,196,0106000020D70800000100000001030000000100000020...,BK,3027101,Brooklyn,E,027101,271.01,BK61,Crown Heights North,...,26686,606,94,15.5,182800,528,30,2.3,28.331,12.138
4,1418,0106000020D7080000010000000103000000010000002C...,QN,4056700,Queens,I,056700,567,QN19,Glendale,...,51544,1502,904,60.2,195300,2119,117,3.5,9.611,19.760


Generate a report of rows with zeros in the median household income

In [13]:
zero_mi_2000 = df2000.loc[df2000['median_hh_income'] == 0]
zero_mi_2000 = zero_mi_2000[~zero_mi_2000['ntaname'].str.contains('park-')]
zero_mi_2000 = zero_mi_2000[~zero_mi_2000['ntaname'].str.contains('Airport')]
zero_mi_2000

,id,geom,borocode,boro_ct2000,boro_name,cdeligibil,ct2000,ctlabel,ntacode,ntaname,...,median_hh_income,tot_housing_units,owner_occupied_housing_units,pct_owner_occupied,median_housing_value,in_labor_force_16up,in_labor_force_unemployed_16up,pct_unemployed_16up,pct_below_poverty_level,pct_educated
1,1781,0106000020D7080000010000000103000000010000002A...,SI,5008900,Staten Island,I,008900,89,SI22,West New Brighton-New Brighton-St. George,...,0,0,0,0.0,0,0,0,0.0,0.000,0.000
46,1473,0106000020D7080000010000000103000000010000001E...,BK,3081200,Brooklyn,I,081200,812,BK60,Prospect Lefferts Gardens-Wingate,...,0,0,0,0.0,0,0,0,0.0,0.000,0.000
132,667,0106000020D70800000100000001030000000100000019...,BX,2004002,Bronx,I,004002,40.02,BX09,Soundview-Castle Hill-Clason Point-Harding Park,...,0,0,0,0.0,0,0,0,0.0,0.000,0.000
235,1716,0106000020D70800000100000001030000000100000041...,BX,2018700,Bronx,I,018700,187,BX63,West Concourse,...,0,0,0,0.0,0,0,0,0.0,0.000,0.000
295,433,0106000020D70800000100000001030000000100000024...,QN,4019100,Queens,I,019100,191,QN31,Hunters Point-Sunnyside-West Maspeth,...,0,0,0,0.0,0,0,0,0.0,0.000,0.977
352,1503,0106000020D708000001000000010300000001000000A9...,BX,2028400,Bronx,I,028400,284,BX37,Van Nest-Morris Park-Westchester Square,...,0,2,1,50.0,0,4,0,0.0,55.696,11.914
459,1688,0106000020D70800000100000001030000000100000064...,QN,4017100,Queens,I,017100,171,QN31,Hunters Point-Sunnyside-West Maspeth,...,0,0,0,0.0,0,0,0,0.0,0.000,0.000
661,1478,0106000020D70800000100000001030000000100000073...,MN,1030100,Manhattan,I,030100,301,MN01,Marble Hill-Inwood,...,0,0,0,0.0,0,0,0,0.0,0.000,0.000
720,1655,0106000020D70800000100000001030000000100000021...,BX,2041000,Bronx,I,041000,410,BX62,Woodlawn-Wakefield,...,0,0,0,0.0,0,0,0,0.0,0.000,0.000
790,879,0106000020D70800000200000001030000000100000011...,QN,4099900,Queens,I,099900,999,QN47,Ft. Totten-Bay Terrace-Clearview,...,0,0,0,0.0,0,0,0,0.0,0.000,0.000


In [36]:
def download_json():
    url = "NYC_Census_Tracts_2000.geojson"
    resp = requests.get(url)
    return resp.json()

def gen_base(geojson):
    '''Generates baselayer of DC ANC map'''
    base = alt.Chart(alt.Data(values=geojson)).mark_geoshape(
        stroke='black',
        strokeWidth=1
    ).encode(
    ).properties(
        width=400,
        height=400
    )
    return base

anc_json = json("NYC_Census_Tracts_2000.geojson")
base_layer = gen_base(geojson=anc_json)
base_layer

TypeError: 'module' object is not callable